In [2]:
import sys

sys.path.insert(0,"../mipnerf/")

In [3]:
from internal import mip

In [4]:
import torch

from nerf_encodings import *
from kilonerf_helpers import *

torch.set_num_interop_threads(32)

dev = "cpu"
dtype = "torch.FloatTensor"

data = np.load('data/tiny_nerf_data.npz')
images = data['images']
poses = data['poses']
focal = data['focal']
H, W = 100,100

i_img = 101 

pose = torch.from_numpy(poses[i_img])

vecs = get_vecs(H,W,focal)#.numpy()
oris, dirs, view_dirs = get_params(vecs,pose)
radii = get_radii(vecs)

/home/mebenstein/micromamba/envs/mipnerf/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import jax.numpy as jnp
import jax
from jax import grad, jit, vmap
from jax import random

jax.config.update('jax_platform_name', 'gpu')
key = random.PRNGKey(0)

In [5]:
%%timeit
t_vals, samples = mip.sample_along_rays(
    key,
    jax.device_put(oris.numpy().astype(np.float16)),
    jax.device_put(dirs.numpy().astype(np.float16)),
    jax.device_put(radii[...,None].numpy().astype(np.float16)),
    128,
    2,
    6,
    True,
    True,
    "cone"
)

22.6 ms ± 417 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
t_vals, samples = mip.sample_along_rays(
    key,
    a,b,c,
    128,
    2,
    6,
    True,
    True,
    "cone"
)

23.9 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
a=jax.device_put(oris.numpy().astype(np.float16))
b=jax.device_put(dirs.numpy().astype(np.float16))
c=jax.device_put(radii[...,None].numpy().astype(np.float16))

In [29]:
jax.profiler.start_trace("tensorboard")

t_vals, samples = mip.sample_along_rays(
    key,
    a,b,c,
    128,
    2,
    6,
    True,
    False,
    "cone"
)

t_vals.block_until_ready()
samples[0].block_until_ready()
samples[1].block_until_ready()

jax.profiler.stop_trace()

In [21]:

jax.profiler.stop_trace()

In [10]:
samples

(DeviceArray([[[-1.0966787e+00, -3.8068056e-02,  1.9722238e+00],
               [-1.0841343e+00, -1.1765003e-02,  1.9579569e+00],
               [-1.0725882e+00,  1.2444854e-02,  1.9448254e+00],
               ...,
               [ 5.9090471e-01,  3.5004444e+00,  5.2917957e-02],
               [ 6.0584974e-01,  3.5317807e+00,  3.5921097e-02],
               [ 6.2200117e-01,  3.5656466e+00,  1.7551899e-02]],
 
              [[-1.0849415e+00, -4.5111060e-02,  1.9701362e+00],
               [-1.0746877e+00, -2.3975968e-02,  1.9586042e+00],
               [-1.0616314e+00,  2.9358864e-03,  1.9439205e+00],
               ...,
               [ 6.2137699e-01,  3.4719753e+00,  5.1121235e-02],
               [ 6.3323712e-01,  3.4964213e+00,  3.7782669e-02],
               [ 6.4865184e-01,  3.5281940e+00,  2.0446777e-02]],
 
              [[-1.0785687e+00, -6.3167214e-02,  1.9746096e+00],
               [-1.0639911e+00, -3.3626795e-02,  1.9584041e+00],
               [-1.0502856e+00, -5.8540106e-